# DATE DIMENSION

In [1]:
from connection_script import connect_databases
import pandas as pd
import locale

In [3]:
_, db_etl = connect_databases()

## TRANSFORM

In [4]:
# Generate range
dim_date = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})
dim_date.tail()

,date
3647,2014-12-27
3648,2014-12-28
3649,2014-12-29
3650,2014-12-30
3651,2014-12-31


In [5]:
dim_date["DateKey"] = dim_date["date"].dt.strftime("%Y%m%d")
dim_date["FullDateAlternateKey"] = dim_date["date"].dt.strftime("%Y-%m-%d")

In [6]:
# Get weekdays
def weekday_index(x):
    days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    if x in days:
        return days.index(x) + 1
    else:
        raise Exception('Invalid day name')
        
dim_date["EnglishDayNameOfWeek"] = dim_date["date"].dt.day_name()
dim_date["EnglishDayNumberOfWeek"] = dim_date["EnglishDayNameOfWeek"].apply(lambda x: weekday_index(x))

# Sort columns
dim_date = dim_date[["date", "DateKey", "FullDateAlternateKey", "EnglishDayNumberOfWeek", "EnglishDayNameOfWeek"]]

In [7]:
# Add rest of day names in different languages
dim_date["SpanishDayNameOfWeek"] = dim_date["date"].dt.day_name('es_CO.UTF8')
dim_date["FrenchDayNameOfWeek"] = dim_date["date"].dt.day_name('fr_FR.UTF8')

In [45]:
def get_fiscal_quarter(quarter):
    quarters = [1, 2, 3, 4]
    fiscal_quarters = [3, 4, 1, 2]

    if quarter in quarters:
        ind = quarters.index(quarter)
        return fiscal_quarters[ind]
    else:
        raise Exception('Invalid quarter')
    

dim_date["DayNumberOfMonth"] = dim_date["date"].dt.day
dim_date["DayNumberOfMonth"] = dim_date["date"].dt.day_of_year
dim_date["WeekNumberOfYear"] = 'pending'
dim_date["EnglishMonthName"] = dim_date["date"].dt.month_name()
dim_date["SpanishMonthName"] = dim_date["date"].dt.month_name('es_CO.UTF8')
dim_date["FrenchMonthName"] = dim_date["date"].dt.month_name('fr_FR.UTF8')
dim_date["MonthNumberOfYear"] = dim_date["date"].dt.month
dim_date["CalendarQuarter"] = dim_date["date"].dt.quarter
dim_date["CalendarYear"] = dim_date["date"].dt.year
dim_date["CalendarSemester"] = dim_date["MonthNumberOfYear"].apply(lambda x: 2 if x > 6 else 1)
dim_date["FiscalQuarter"] = dim_date["CalendarQuarter"].apply(lambda x: get_fiscal_quarter(x))



In [46]:
dim_date

,date,DateKey,FullDateAlternateKey,EnglishDayNumberOfWeek,EnglishDayNameOfWeek,SpanishDayNameOfWeek,FrenchDayNameOfWeek,DayNumberOfMonth,WeekNumberOfYear,EnglishMonthName,SpanishMonthName,FrenchMonthName,MonthNumberOfYear,CalendarQuarter,CalendarYear,CalendarSemester,FiscalQuarter
0,2005-01-01,20050101,2005-01-01,7,Saturday,Sã¡bado,Samedi,1,pending,January,Enero,Janvier,1,1,2005,1,3
1,2005-01-02,20050102,2005-01-02,1,Sunday,Domingo,Dimanche,2,pending,January,Enero,Janvier,1,1,2005,1,3
2,2005-01-03,20050103,2005-01-03,2,Monday,Lunes,Lundi,3,pending,January,Enero,Janvier,1,1,2005,1,3
3,2005-01-04,20050104,2005-01-04,3,Tuesday,Martes,Mardi,4,pending,January,Enero,Janvier,1,1,2005,1,3
4,2005-01-05,20050105,2005-01-05,4,Wednesday,Miã©rcoles,Mercredi,5,pending,January,Enero,Janvier,1,1,2005,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3647,2014-12-27,20141227,2014-12-27,7,Saturday,Sã¡bado,Samedi,361,pending,December,Diciembre,Dã©cembre,12,4,2014,2,2
3648,2014-12-28,20141228,2014-12-28,1,Sunday,Domingo,Dimanche,362,pending,December,Diciembre,Dã©cembre,12,4,2014,2,2
3649,2014-12-29,20141229,2014-12-29,2,Monday,Lunes,Lundi,363,pending,December,Diciembre,Dã©cembre,12,4,2014,2,2
3650,2014-12-30,20141230,2014-12-30,3,Tuesday,Martes,Mardi,364,pending,December,Diciembre,Dã©cembre,12,4,2014,2,2
